# Case Study: Black Pavilion with Conic Skyligths

In [1]:
# OS functionalities (to deal with files)
import os

# Enhanced Iteration capabilities (to use cross-products)
import itertools

# Data processing packages
import pandas as pd
import numpy as np

## Global Setup for Processing Data

In this section, we define all the global constants that will be used thourought the script. By defining them up here, we are making this notebook purely parametric and reusable for other projects (as long as the folder structure remains the same).  

In [2]:
# Folders Structure
base_folder = os.getcwd()
results_folder = os.path.join(base_folder, "algorithms")
output_folder = "./output/"

# CSV File Configuration
has_header=True
files_sep= ","
file_extension = 'csv'

# Optimization Settings
runs = [1, 2, 3]
nruns = len(runs)
max_evals = 600

## Problem Definition (in the files) 
### Variables
n_skylights = 7
max_dist = 8
center_y = 9
radius_m = 10
radius_M = 11
skylight_material = 12
curtain_wall_material = 13
vars_cols = [n_skylights, max_dist, center_y, radius_m, radius_M, skylight_material, curtain_wall_material]

### Objectives  
material_cost = 20
sUDI = 21
objs_cols = [material_cost, sUDI]

feasible_col = 19

relevant_cols = vars_cols + objs_cols + [feasible_col]

## Multi-Objective Optimization Algorithms
### Metaheuristics
pop_size = 40
metaheuristics = ["SMPSO", "NSGAII", "SPEA2"]

### Model Based (or metamodel)
metamodels_base = ['GPR'] 
metamodels_strategies = ["SMPSO", 'NSGAII', "SPEA2"]
metamodels_algorithms = [f"{b}_{s}" for (b, s) in itertools.product(metamodels_base, metamodels_strategies)]

all_algorithms = metaheuristics + metamodels_algorithms
n_algorithms = len(all_algorithms)

### Filenames with the results 
filenames = [f"{a}_results_0{r}.{file_extension}" for (r, a) in itertools.product(runs, all_algorithms)]
filenames

['SMPSO_results_01.csv',
 'NSGAII_results_01.csv',
 'SPEA2_results_01.csv',
 'GPR_SMPSO_results_01.csv',
 'GPR_NSGAII_results_01.csv',
 'GPR_SPEA2_results_01.csv',
 'SMPSO_results_02.csv',
 'NSGAII_results_02.csv',
 'SPEA2_results_02.csv',
 'GPR_SMPSO_results_02.csv',
 'GPR_NSGAII_results_02.csv',
 'GPR_SPEA2_results_02.csv',
 'SMPSO_results_03.csv',
 'NSGAII_results_03.csv',
 'SPEA2_results_03.csv',
 'GPR_SMPSO_results_03.csv',
 'GPR_NSGAII_results_03.csv',
 'GPR_SPEA2_results_03.csv']

In [3]:
# Sanity check: verify the file names are what we expected
filenames[::3]

['SMPSO_results_01.csv',
 'GPR_SMPSO_results_01.csv',
 'SMPSO_results_02.csv',
 'GPR_SMPSO_results_02.csv',
 'SMPSO_results_03.csv',
 'GPR_SMPSO_results_03.csv']

## Input/Output (IO) Methods 

In this subsection we create the methods that will be responsible for loading the data from the files. To manipulate the data, we will use pandas.DataFrame data structure. This can be easily manipulated and different statistics can be computed on top of these abstractions.

In [4]:
def load_results(filenames, base_folder=results_folder, 
                 has_header=True, keep_header=False, 
                 sep=files_sep, usecols=relevant_cols,  
                 max_lines=max_evals):
    """Loads the data from the specified `base_folder` using the `filenames`.
    It assumes the filenames
    """
    read_args = { 
        "header": 'infer' if keep_header else None, 
        "sep": sep,
        "usecols": usecols if usecols else None,
        "skiprows": 1 if has_header and not keep_header else 0,
    }
    filepaths = [os.path.join(base_folder, f) for f in filenames]        
    if max_lines:
        return [pd.read_csv(f, **read_args)[0:max_lines] for f in filepaths]
    else:
        return [pd.read_csv(f, **read_args) for f in filepaths]

In [5]:
# Always confirm whether the results are according to what you expected
examples = load_results(filenames, max_lines=max_evals)
examples[0].head()

,7,8,9,10,11,12,13,19,20,21
0,6,1484,166,6,16,0,1,False,9.223372e+18,9.223372e+18
1,3,1404,204,2,14,2,3,True,1.390607e+04,-0.000000e+00
2,7,1345,30,8,15,3,0,False,9.223372e+18,9.223372e+18
3,6,2134,187,7,2,2,3,False,9.223372e+18,9.223372e+18
4,3,1737,84,3,9,1,2,False,9.223372e+18,9.223372e+18


In [6]:
list(filter(lambda x: x > max_evals, [len(e) for e in examples]))

[]

In [7]:
def get_run_indices(dfs, run, n_algorithms=n_algorithms):
    """Returns the dataframes that correspond to the specified `run`. 
    This assumes that the dataframes are read per run, that means that 
    if we run two algorithms (SMPSO and SPEA2) for 3 runs each, this 
    method assumes that it was read in the following order:
    
    > SMPSO_run1
    > SPEA2_run1
    > SMPSO_run2
    > SPEA2_run2
    > SMPSO_run3
    > SPEA2_run3
    
    In that case, this invocation `get_run_indices(dfs, 1, n_algorithms=2)`
    will return: 
        dfs[0:2] 
    """
    run -= 1
    return dfs[run*n_algorithms:(run+1)*n_algorithms]

### Pareto Dominance Methods
This section contains methods related to the Pareto optimality (or [Pareto Efficiency](https://en.wikipedia.org/wiki/Pareto_efficiency)).

In [8]:
# **IMPORTANT NOTE**: This function assumes that your problem is a minimization problem for every objective dimension.
def weakly_dominates(v0, v1):
    """Computes whether v0 dominates v1, i.e., whether at least one objective
    is better (in this case, smaller) than some other)
    """
    return np.all(v0 <= v1) and np.any(v0 < v1)
    
# Sanity Check (:
print("(Expected) True   (Obtained)", weakly_dominates(np.array([1, 1]), np.array([2, 2])))
print("(Expected) True   (Obtained)", weakly_dominates(np.array([2, 1]), np.array([2, 2])))
print("(Expected) False  (Obtained)", weakly_dominates(np.array([2, 2]), np.array([1, 1]))) 
print("(Expected) True   (Obtained)", weakly_dominates(np.array([1, 2]), np.array([2, 2]))) 
print("(Expected) False  (Obtained)", weakly_dominates(np.array([1, 3]), np.array([3, 1])))
print("(Expected) False  (Obtained)", weakly_dominates(np.array([3, 1]), np.array([1, 3]))) 
print("(Expected) False  (Obtained)", weakly_dominates(np.array([1, 1]), np.array([1, 1])))  

(Expected) True   (Obtained) True
(Expected) True   (Obtained) True
(Expected) False  (Obtained) False
(Expected) True   (Obtained) True
(Expected) False  (Obtained) False
(Expected) False  (Obtained) False
(Expected) False  (Obtained) False


In [9]:
def get_non_dominated(V, dominance=weakly_dominates):
    """Computes the optimal and non-optimal solutions. 
    Optimal solutions are called non-dominated and non-optimal 
    solutions are called denominated."""
    nsols, nobjs = V.shape
    
    dominated = np.zeros((nsols, 1))
    dominated_by = np.zeros((nsols, 1))
    for i in range(nsols):
        for j in range(nsols):
            if i != j:
                if dominance(V[j], V[i]):
                    dominated[i] = 1
                    dominated_by[i] = j 
                    break
                    
    return dominated, dominated_by

In [10]:
def add_isdominated_cols(d, cols=objs_cols):
    """Adds to the provided dataframe columns for Pareto optimal solution."""
    df_copy = d.copy()
    A = np.array(df_copy[cols])
    B, C = get_non_dominated(A)
    df_copy["isDominated"] = pd.DataFrame(B, columns=["isDominated"])
    df_copy["dominatedBy"] = pd.DataFrame(C, columns=["dominatedBy"])
    print(df_copy["isDominated"].value_counts())
    return df_copy

In [11]:
def get_combined_PF(dfs, drop_cols=relevant_cols, objs_cols=objs_cols):
    """Computes the combined Pareto front based on a set of input dataframes"""
    all_data = pd.concat(dfs)
    if drop_cols:
        all_data = all_data.drop_duplicates(drop_cols)
    all_data = all_data.reset_index()
    return add_isdominated_cols(all_data, cols=objs_cols)

### General Methods

This section contains general purpose methods that can be used in your scripts.

In [12]:
def broadcasting_multi(df, cols, value):
    cols = cols if isinstance(cols, (list, tuple)) else [cols]
    copy_df = df.copy()
    for col in cols:
        copy_df[col] = df[col] * value
    return copy_df

In [13]:
def broadcasting_div(df, cols, value):
    cols = cols if isinstance(cols, (list, tuple)) else [cols]
    copy_df = df.copy()
    for col in cols:
        copy_df[col] = df[col] / value
    return copy_df

In [14]:
def get_symmetric(df, cols):
    """Computes the symmetric value of the provided `cols` and returns a 
    copy of the original dataframe where the values of the specified `cols`
    are symmetric."""
    return broadcasting_multi(df, cols, -1)

In [15]:
def unscale(df, cols, minimum, step):
    """Computes the real value used for the variables during a discrete optimization process."""
    cols = cols if isinstance(cols, (list, tuple)) else [cols]
    copy_df = df.copy()
    for col in cols:
        copy_df[col] = minimum + step * df[col]
    return copy_df

In [16]:
def drop_by_value(dfs, col, value):
    """Drops the solutions from the provided dataframes based on a column and a value, 
    changing them inplace."""
    for df in dfs:      
        unfeasible_sols = df.loc[df[col]==value]
        df.drop(unfeasible_sols.index, inplace=True)
        df.index = range(len(df))
    return dfs

In [17]:
def drop_unfeasibles(dfs):
    """Drops the unfeasible solutions from the provided dataframes, 
    changing them inplace."""
    return drop_by_value(dfs, feasible_col, False)

In [18]:
def unfeasible_ratio_iter(pop_size=pop_size):
    """Computes the unfeasible ratio per `pop_size`."""
    results = []
    for i in range(0, len(df), pop_size):
        results.append(df[df[i:i + pop_size] == 'false']['feasible'].count() / pop_size)
    iteration = pd.Series(data = range(1, int(len(df) / pop_size) + 1))
    ratio = pd.Series(data = results)
    ratio_iter = {'Iteration': iteration, 'Ratio Unfeasible': ratio} 
    return pd.DataFrame(ratio_iter)

In [19]:
def get_last_value_iter(df, col_name='hypervolume', iter_size=pop_size):
    """Iterates the values in the `df` in slices of `iter_size` and 
    gets the last value.
    """
    return df[iter_size-1:len(df):iter_size][col_name]

In [20]:
def indicators_results(results_runs):
    """Computes the statistics mean and standard deviation for the specified runs. 
    
    It assumes the `results_run` are organized as follows: 
        results_runs = [indicators_run1, indicators_run2, indicators_run3], 
    where: 
        indicators_run{i} = [indicators_alg_1_run_{i}, ..., indicators_alg_m_run_{i}]
    """
    average_res = [] 
    std_res = [] 
    
    for algorithm_i in range(n_algorithms):
        # Get the same algorithm from all runs (assuming they were collected in the same way)
        algorithm_results = [r[algorithm_i] for r in results_runs] 
        average_res += [pd.concat(algorithm_results, axis=1).mean(axis=1)]
        std_res += [pd.concat(algorithm_results, axis=1).std(axis=1)]
        
    return average_res, std_res

### Data Visualization

In this section, we visualize the data with visual means. 

In [21]:
# Visualization Framework
import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib inline
init_notebook_mode(connected=True)

try:
    import plotly.plotly as py
    plotly.tools.set_credentials_file(username='username', api_key='api_key')
except: 
    import chart_studio
    import chart_studio.plotly as py
    chart_studio.tools.set_credentials_file(username='username', api_key='api_key')
    
# Print plotly's version
plotly.__version__

'5.1.0'

Let us create the functions that create the graphs

In [22]:
def get_colors(n, colorscale="viridis"): 
    colors = plt.get_cmap(colorscale).colors
    colors_idx = np.linspace(0, len(colors)-1, n, dtype=int)
    
    colors = [colors[idx] for idx in colors_idx]
    colors_str = [f"rgb({r}, {g}, {b})" for (r, g, b) in colors]
    return colors_str

In [23]:
def scatter(data, x=None, y=None, names=all_algorithms, 
            colorscale="viridis", colors=None, 
            mode="markers", marker_size=5.5, ln_width=1.5, 
            layout=None):
    
    def get_x_y(d): 
        if isinstance(d, pd.Series):
            return np.array(d.index) + 1, d.values
        elif isinstance(d, pd.DataFrame):
            return d[x], d[y]
        else: 
            return np.arange(len(d)), d      
        
    # ----------------------------------------------
    # Normalize input data
    # ----------------------------------------------
    data = data if isinstance(data, (list, tuple)) else [data]
    colors = get_colors(len(data), colorscale) if colorscale else colors
    
    # ----------------------------------------------
    # Determine the data types of provided inputs
    # ----------------------------------------------
    x_data = []
    y_data = []
    for d in data:
        xx, yy = get_x_y(d)
        x_data += [xx]
        y_data += [yy]

    
    traces = []
    for (i, (x,y)) in enumerate(zip(x_data, y_data)):
        traces += [
            go.Scatter(
                x = x,
                y = y,
                mode = mode,
                name = names[i],
                marker = dict(
                    # Markers size
                    size = marker_size,
                    color=colors[i],
                ),
                line=dict(
                    width=ln_width,
                    color=colors[i]
            )
            )]
    
    kwargs = {} if not layout else {"layout": layout}
    fig = go.Figure(data=traces, **kwargs)
    return py.iplot(fig, filename='simple_scatter')

#### Default Layout

In [24]:
# Default layout for the pareto fronts graphs
layout = go.Layout(
    template="plotly_white",
    autosize=True,
    legend=dict(
        orientation='h'
    ),
    # Define axis
    xaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.'
    ),
    yaxis=dict(
        autorange=True,        
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.'
    )
)

#### Pareto Front

This section contains different functions that explore dataframes having information about the non-dominated and the dominated solutions.

In [25]:
def create_pf(pf, name, x, y, nd_color='rgb(0,0,255)', ln_width=1.5, marker_size=5.5, d_color=None): 
    traces = []
    
    # Create the non dominated trace (in a different color, as specified in *nd_color*)
    x_pf = pf[pf['isDominated'] == 0][x] 
    y_pf = pf[pf['isDominated'] == 0][y]
    x_pf, y_pf = zip(*sorted(zip(x_pf, y_pf)))
    
    # Get information about the objectives and variables - relevant for an iteractive PF
    info = []
    info.extend(vars_cols)
    info.extend(objs_cols)
    info_pf = np.array(pf[pf['isDominated'] == 0][info])
    info_npf = np.array(pf[pf['isDominated'] == 1][info])
    
    traces += [
        go.Scatter(
            x = x_pf,
            y = y_pf,
            mode = 'lines+markers',
            name = name + " NonDominated",
            customdata = info_pf,
            opacity=1,

            # Layout do marker
            marker=dict(
                color=nd_color,
                size=marker_size
            ),
            line=dict(
                color=nd_color,
                width=ln_width
            )
        )]
    
    if d_color:
        x_npf = pf[pf['isDominated'] == 1][x]
        y_npf = pf[pf['isDominated'] == 1][y]
        
        # Create the dominated trace (in a different color, as specified in *d_color*)
        traces +=[
            go.Scatter(
                x = x_npf,
                y = y_npf,
                mode = 'markers',
                name = name + " Dominated",
                customdata = info_npf,
                opacity=0.7,

                # Layout do Marker
                marker=dict(
                    #color = d_color,
                    color = nd_color,
                    size=marker_size,
                )
            )]

    return traces

In [26]:
def get_traces(pfs, x, y, draw_dominated=True,
               names=all_algorithms, colorscale='viridis', colors=None,
               tpf=None, tpf_name="Combined_PF", tpf_color='rgb(0,0,0)', 
               layout=layout):
    
    pfs = pfs if isinstance(pfs, (list, tuple)) else [pfs]
    names = names if isinstance(names, (list, tuple)) else [names]
    n_pfs = len(pfs)
    colors = get_colors(n_pfs, colorscale) if colorscale else colors
    
    traces = []
    
    if tpf is not None:
        traces += create_pf(pf=tpf, name=tpf_name, x=x, y=y, ln_width=4, marker_size=10, nd_color=tpf_color)
    
    for (i, pf) in enumerate(pfs):
        d_color = colors[i] if draw_dominated else None
        traces += create_pf(pf=pf, name=names[i], x=x, y=y, nd_color=colors[i], d_color=d_color)

    fig = go.Figure(data=traces, layout=layout)
    return traces

In [27]:
def create_pfs(pfs, x, y, draw_dominated=True,
               names=all_algorithms, colorscale='viridis', colors=None,
               tpf=None, tpf_name="Combined_PF", tpf_color='rgb(0,0,0)', 
               layout=layout):
    
    traces = get_traces(pfs, x, y, draw_dominated, names, colorscale, colors, tpf, tpf_name, tpf_color, layout)
    
    fig = go.Figure(data=traces, layout=layout)
    return py.iplot(fig, filename='algorithms_pfs_per_run')

---

## Analysis


### Pareto Front Layout

In [28]:
layout_BP = go.Layout(
    template="plotly_white",
#     template='ggplot2',
    autosize=False,
    # Define plot size
    width=900, 
    height=600,
    # Legend Position
    legend=dict(
        orientation='h',
        x=-0.01,
        y=-0.2
    ),
    # Define axis
    xaxis=dict(
        title="Cost [€]",
        range=[10000, 19500],
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.'
    ),
    yaxis=dict(
        title="sUDI [%]",
        range=[0, 50],    
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.',
        dtick=10
    )
)

### Read Algorithms

In [29]:
# Read algorithms  
dfs = load_results(filenames)

In [30]:
dfs = drop_unfeasibles(dfs)

In [31]:
# Sanity check!!
dfs[0].head()

,7,8,9,10,11,12,13,19,20,21
0,3,1404,204,2,14,2,3,True,13906.074007,-0.0
1,5,1657,136,5,7,2,1,True,14789.909373,-14.0
2,2,1282,146,2,4,1,0,True,11638.584709,-0.0
3,7,1867,197,5,6,1,1,True,16689.478473,-0.0
4,3,1835,96,0,1,1,4,True,9967.813004,-0.0


In [32]:
# Run only if you want to remove the cells with sUDI = 0
dfs = drop_by_value(dfs, sUDI, 0)

In [33]:
# Sanity check!!
dfs[0].head()

,7,8,9,10,11,12,13,19,20,21
0,5,1657,136,5,7,2,1,True,14789.909373,-14.0
1,5,1619,145,4,9,3,2,True,14756.316295,-23.0
2,5,1719,143,5,7,2,1,True,14789.909373,-14.0
3,5,1581,163,5,9,3,3,True,16375.010171,-26.0
4,5,1713,125,3,7,3,3,True,15269.361347,-1.0


In [34]:
# Compute non_dominated_solutions (per run)
pfs = [add_isdominated_cols(df) for df in dfs]

1.0    106
0.0     12
Name: isDominated, dtype: int64
1.0    162
0.0     16
Name: isDominated, dtype: int64
1.0    81
0.0     7
Name: isDominated, dtype: int64
1.0    169
0.0      5
Name: isDominated, dtype: int64
1.0    380
0.0      9
Name: isDominated, dtype: int64
1.0    158
0.0      7
Name: isDominated, dtype: int64
1.0    156
0.0     13
Name: isDominated, dtype: int64
1.0    219
0.0     16
Name: isDominated, dtype: int64
1.0    10
0.0     6
Name: isDominated, dtype: int64
1.0    206
0.0     10
Name: isDominated, dtype: int64
1.0    426
0.0     14
Name: isDominated, dtype: int64
1.0    155
0.0      8
Name: isDominated, dtype: int64
1.0    92
0.0     8
Name: isDominated, dtype: int64
1.0    237
0.0     13
Name: isDominated, dtype: int64
1.0    113
0.0     13
Name: isDominated, dtype: int64
1.0    157
0.0     12
Name: isDominated, dtype: int64
1.0    280
0.0     14
Name: isDominated, dtype: int64
1.0    149
0.0      8
Name: isDominated, dtype: int64


In [35]:
# Sanity check!!
pfs[0].head()

,7,8,9,10,11,12,13,19,20,21,isDominated,dominatedBy
0,5,1657,136,5,7,2,1,True,14789.909373,-14.0,1.0,1.0
1,5,1619,145,4,9,3,2,True,14756.316295,-23.0,1.0,19.0
2,5,1719,143,5,7,2,1,True,14789.909373,-14.0,1.0,1.0
3,5,1581,163,5,9,3,3,True,16375.010171,-26.0,1.0,5.0
4,5,1713,125,3,7,3,3,True,15269.361347,-1.0,1.0,0.0


In [36]:
# Run this cell only once!!!
pfs = [unscale(pf, max_dist, 0.14, 0.01) for pf in pfs]
pfs = [unscale(pf, center_y, 0.44, 0.01) for pf in pfs]
pfs = [unscale(pf, radius_m, 0.20, 0.10) for pf in pfs]
pfs = [unscale(pf, radius_M, 0.30, 0.10) for pf in pfs]

In [37]:
# Sanity check!!
pfs[0][vars_cols].head()

,7,8,9,10,11,12,13
0,5,16.71,1.80,0.7,1.0,2,1
1,5,16.33,1.89,0.6,1.2,3,2
2,5,17.33,1.87,0.7,1.0,2,1
3,5,15.95,2.07,0.7,1.2,3,3
4,5,17.27,1.69,0.5,1.0,3,3


In [38]:
pfs[0][pfs[0][sUDI] == 0]

,7,8,9,10,11,12,13,19,20,21,isDominated,dominatedBy


In [39]:
# Computes combined Pareto Front (optimal solutions found from all the algorithms, all the runs)
combined_pf = get_combined_PF(dfs, drop_cols=relevant_cols)

1.0    3400
0.0      21
Name: isDominated, dtype: int64


In [40]:
# Since sUDI is actually a maximization, let us use the symmetric operation 
pfs = [get_symmetric(pf, sUDI) for pf in pfs]
combined_pf = get_symmetric(combined_pf, sUDI)

### Plot for all obtained solution (1 single plot for all algorithms, all runs) 


In [41]:
create_pfs(pfs, x=material_cost, y=sUDI, tpf=combined_pf, names=filenames, 
           colorscale='plasma', draw_dominated=False, layout=layout_BP)

In [42]:
create_pfs(pfs, x=material_cost, y=sUDI, tpf=combined_pf, names=filenames,
           colorscale='plasma', draw_dominated=True, layout=layout_BP)

### Plot algorithms per run (3 plot one for each run) 

In [43]:
# Get Run 1 
dfs_run_1 = get_run_indices(dfs, 1)
pfs_run_1 = [add_isdominated_cols(df) for df in dfs_run_1]
combined_PF_run_1 = get_combined_PF(pfs_run_1)

1.0    106
0.0     12
Name: isDominated, dtype: int64
1.0    162
0.0     16
Name: isDominated, dtype: int64
1.0    81
0.0     7
Name: isDominated, dtype: int64
1.0    169
0.0      5
Name: isDominated, dtype: int64
1.0    380
0.0      9
Name: isDominated, dtype: int64
1.0    158
0.0      7
Name: isDominated, dtype: int64
1.0    1084
0.0      20
Name: isDominated, dtype: int64


In [44]:
# Get Run 2 
dfs_run_2 = get_run_indices(dfs, 2)
pfs_run_2 = [add_isdominated_cols(df) for df in dfs_run_2]
combined_PF_run_2 = get_combined_PF(pfs_run_2)

1.0    156
0.0     13
Name: isDominated, dtype: int64
1.0    219
0.0     16
Name: isDominated, dtype: int64
1.0    10
0.0     6
Name: isDominated, dtype: int64
1.0    206
0.0     10
Name: isDominated, dtype: int64
1.0    426
0.0     14
Name: isDominated, dtype: int64
1.0    155
0.0      8
Name: isDominated, dtype: int64
1.0    1209
0.0      20
Name: isDominated, dtype: int64


In [45]:
# Get Run 3 
dfs_run_3 = get_run_indices(dfs, 3)
pfs_run_3 = [add_isdominated_cols(df) for df in dfs_run_3]
combined_PF_run_3 = get_combined_PF(pfs_run_3)

1.0    92
0.0     8
Name: isDominated, dtype: int64
1.0    237
0.0     13
Name: isDominated, dtype: int64
1.0    113
0.0     13
Name: isDominated, dtype: int64
1.0    157
0.0     12
Name: isDominated, dtype: int64
1.0    280
0.0     14
Name: isDominated, dtype: int64
1.0    149
0.0      8
Name: isDominated, dtype: int64
1.0    1077
0.0      14
Name: isDominated, dtype: int64


In [46]:
# Compute combined Pareto Front for run 1
pfs_run_1 = [get_symmetric(df, sUDI) for df in pfs_run_1]
combined_PF_run_1 = get_symmetric(combined_PF_run_1, sUDI)

In [47]:
# Compute combined Pareto Front for run 2
pfs_run_2 = [get_symmetric(df, sUDI) for df in pfs_run_2]
combined_PF_run_2 = get_symmetric(combined_PF_run_2, sUDI)

In [48]:
# Compute combined Pareto Front for run 3
pfs_run_3 = [get_symmetric(df, sUDI) for df in pfs_run_3]
combined_PF_run_3 = get_symmetric(combined_PF_run_3, sUDI)

In [49]:
# Get All Runs
# Before running this run the previous cells (the ones that define dfs_run_{i})
pfs_all_runs = []
for i in range(n_algorithms):
    pf=get_combined_PF([dfs_run_1[i], dfs_run_2[i], dfs_run_3[i]])
    pf=get_symmetric(pf, sUDI)
    pfs_all_runs.append(pf)

1.0    369
0.0     12
Name: isDominated, dtype: int64
1.0    642
0.0     18
Name: isDominated, dtype: int64
1.0    215
0.0     12
Name: isDominated, dtype: int64
1.0    535
0.0     15
Name: isDominated, dtype: int64
1.0    1103
0.0      17
Name: isDominated, dtype: int64
1.0    476
0.0      9
Name: isDominated, dtype: int64


#### Run 1

In [50]:
# Create plot with combined pareto front retrieved only from all algorithms of run 1
create_pfs(pfs_run_1, x=material_cost, y=sUDI, tpf=combined_PF_run_1,
           names=all_algorithms, draw_dominated=True, layout=layout_BP)

In [51]:
# Create plot with combined pareto front retrieved from all runs
create_pfs(pfs_run_1, x=material_cost, y=sUDI, tpf=combined_pf, 
           names=all_algorithms, draw_dominated=True, layout=layout_BP)

#### Run 2

In [52]:
# Create plot with combined pareto front retrieved only from all algorithms of run 2
create_pfs(pfs_run_2, x=material_cost, y=sUDI, tpf=combined_PF_run_2, 
           names=all_algorithms, draw_dominated=True, layout=layout_BP)

In [53]:
# Create plot with combined pareto front retrieved from all runs
create_pfs(pfs_run_2, x=material_cost, y=sUDI, tpf=combined_pf, 
           names=all_algorithms, draw_dominated=True, layout=layout_BP)

#### Run 3

In [54]:
# Create plot with combined pareto front retrieved only from all algorithms of run 3
create_pfs(pfs_run_3, x=material_cost, y=sUDI, tpf=combined_PF_run_3, 
           names=all_algorithms, draw_dominated=True, layout=layout_BP)

In [55]:
# Create plot with combined pareto front retrieved from all runs
create_pfs(pfs_run_3, x=material_cost, y=sUDI, tpf=combined_pf, 
           names=all_algorithms, draw_dominated=True, layout=layout_BP)

#### All Runs

In [56]:
# Create plot with one PF per algorthim, using the information of the 3 runs
create_pfs(pfs_all_runs, x=material_cost, y=sUDI, tpf=None, names=all_algorithms, 
           colorscale=None,
           colors=['#93e0ed', '#307382', '#2fcce0', '#042780', 
                   '#fcb447', '#ff8000', '#db3f30', 
                   '#de7ec9', '#8a39db', '#db30b9'],
           draw_dominated=False, layout=layout_BP)

In [57]:
# Create plot with one PF per algorthim, using the information of the 3 runs and with the combined PF
create_pfs(pfs_all_runs, x=material_cost, y=sUDI, tpf=combined_pf, tpf_color='#3b3b3b',
           names=all_algorithms,
           colorscale=None,
           colors=['#E6739F', 
                   '#CC0E74', 
                   '#790C5A',
                   '#a7d49f',
                   '#75daad',
                   '#216353'],
           draw_dominated=False, layout=layout_BP)